In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob


def recursive_file_gen(my_root_dir):
    for root, dirs, files in os.walk(my_root_dir):
        for file in files:
            yield os.path.join(root, file)

In [2]:
last_saturday=datetime.datetime.now().date()-datetime.timedelta(days=datetime.datetime.now().date().weekday()+2)

First_week_ending_Q4_2018=datetime.date(2018,11,10)
Q4_2018_weeks=[First_week_ending_Q4_2018+datetime.timedelta(days=x*7) for x in range(13)]

Q4_2018_weeks=[x for x in Q4_2018_weeks if x <=last_saturday]

Q4_2017_weeks=[x-datetime.timedelta(days=52*7) for x in Q4_2018_weeks]
len_Q4_weeks=len(Q4_2018_weeks)

write_folder="/home/jian/celery/Quadrants_Dashboard/Checking_Daily_Data_"+str(last_saturday)+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

In [3]:
sales_all=pd.read_csv("/home/jian/BiglotsCode/outputs/combined_sales_long_"+str(last_saturday)+".csv",dtype=str,usecols=['location_id','sales','transactions','week_end_date'])
sales_all['sales']=sales_all['sales'].astype(float)
sales_all['transactions']=sales_all['transactions'].astype(float).astype(int)
sales_all=sales_all[sales_all['sales']>0]
sales_all_Q4_2017=sales_all[sales_all['week_end_date'].isin([str(x) for x in Q4_2017_weeks])]
sales_all_Q4_2018=sales_all[sales_all['week_end_date'].isin([str(x) for x in Q4_2018_weeks])]

sales_all_Q4_2018=sales_all_Q4_2018.rename(columns={"sales":"sales_weekly_data","transactions":"trans_weekly_data"})
sales_all_Q4_2017=sales_all_Q4_2017.rename(columns={"sales":"sales_weekly_data","transactions":"trans_weekly_data"})

sales_all_Q4_weekly=sales_all_Q4_2018.append(sales_all_Q4_2017)
sales_all_Q4_weekly['week_end_date']=sales_all_Q4_weekly['week_end_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())

In [4]:
# Checking for the recent weeks that haven't moved to the folder
possible_recent_folders=["/home/jian/BigLots/MediaStorm_"+str(x)+"/" for x in Q4_2018_weeks]
recent_daily_list=[]
for dirc in possible_recent_folders:
    list_recent=[x for x in list(recursive_file_gen(dirc)) if ("aily" in x) & (".txt" in x) ]
    recent_daily_list=recent_daily_list+list_recent
recent_daily_df=pd.DataFrame({"path":recent_daily_list,"date":[datetime.datetime.strptime(x.split("MediaStormDailySales")[1][:8],"%Y%m%d").date()-datetime.timedelta(days=3) for x in recent_daily_list]},index=[x for x in range(len(recent_daily_list))])


In [5]:
list_1_after_201806=[x for x in list(recursive_file_gen("/home/jian/BigLots/2018_by_weeks/")) if ("aily" in x) & (".txt" in x) ]
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-2].split("_")[1],"%Y-%m-%d").date() for x in list_1_after_201806]
df_1_after_201806=pd.DataFrame({"date":folder_date,"path":list_1_after_201806},index=[x for x in range(len(list_1_after_201806))])
df_1_after_201806['date'].apply(lambda x: x.weekday()).unique()
df_1_after_201806=df_1_after_201806.sort_values("date").reset_index()
del df_1_after_201806['index']

list_2_before_201806=list(recursive_file_gen("/home/jian/BigLots/hist_daily_data/"))
folder_date=[datetime.datetime.strptime(x.split("/")[len(x.split("/"))-1].split("_")[3].split(".")[0],"%Y-%m-%d").date() for x in list_2_before_201806]
df_2_before_201806=pd.DataFrame({"date":folder_date,"path":list_2_before_201806},index=[x for x in range(len(list_2_before_201806))])
df_2_before_201806['date'].apply(lambda x: x.weekday()).unique()
df_2_before_201806=df_2_before_201806.sort_values('date').reset_index()
del df_2_before_201806['index']
df_2_before_201806['date'].apply(lambda x: x.weekday()).unique()

df_Q4_2017=df_2_before_201806[df_2_before_201806['date'].isin(Q4_2017_weeks)]
df_Q4_2018=df_1_after_201806[df_1_after_201806['date'].isin(Q4_2018_weeks)]
df_Q4_2018=df_Q4_2018.append(recent_daily_df)

In [6]:
def add_week_end_dt(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=5-x.weekday())
    return y

In [9]:
agg_daily_data=pd.DataFrame()
for file_path in df_Q4_2017['path'].tolist()+df_Q4_2018['path'].tolist():
    df=pd.read_table(file_path,dtype=str,sep="|",usecols=['location_id','customer_id_hashed','transaction_id','transaction_dt','subclass_transaction_amt'])
    df['subclass_transaction_amt']=df['subclass_transaction_amt'].astype(float)
    df['transaction_dt']=df['transaction_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
    df['week_end_date']=df['transaction_dt'].apply(lambda x: add_week_end_dt(x))

    df_sales=df.groupby(['location_id','week_end_date'])['subclass_transaction_amt'].sum().to_frame().reset_index().rename(columns={"subclass_transaction_amt":"sales_daily_data"})
    del df['subclass_transaction_amt']
    df=df.drop_duplicates()
    df_trans=df.groupby(['location_id','week_end_date'])['transaction_id'].count().to_frame().reset_index().rename(columns={"transaction_id":"trans_daily_data"})
    df=pd.merge(df_sales,df_trans,on=['location_id','week_end_date'],how="outer")
    agg_daily_data=agg_daily_data.append(df)
    print(datetime.datetime.now())
    

    
    

2018-12-20 16:57:10.802541
2018-12-20 17:02:10.646637
2018-12-20 17:09:49.943456
2018-12-20 17:19:13.934604
2018-12-20 17:25:47.101760
2018-12-20 17:32:55.778431
2018-12-20 17:36:08.383100
2018-12-20 17:40:46.845579
2018-12-20 17:47:38.363430
2018-12-20 17:53:53.176310
2018-12-20 18:00:53.051423
2018-12-20 18:07:28.365525


In [10]:
compared_df=pd.merge(sales_all_Q4_weekly,agg_daily_data,on=['location_id','week_end_date'],how="outer")
compared_df.to_csv(write_folder+"BL_QC_Q4_daily_data.csv",index=False)

In [11]:
write_folder

'/home/jian/celery/Quadrants_Dashboard/Checking_Daily_Data_2018-12-15/'